In [ ]:
import os
import logging
import json
from pathlib import Path

from datasets import load_dataset, DatasetDict

data_dir = Path("/opt/ml/final/data/ko-ja")
folder_list = os.listdir(data_dir)

dataset_dict = dict()

for folder in folder_list:
    data = load_dataset("csv", data_files=[str(p) for p in data_dir.joinpath(folder).glob("*.csv")])
    dataset_dict[folder] = data['train']

raw_dataset = DatasetDict(dataset_dict)
train_set = raw_dataset["Training"]
valid_set = raw_dataset["Validation"]

Using custom data configuration default-4568c14414cede3a


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/default-4568c14414cede3a/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-13d6237d81cc2388


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/default-13d6237d81cc2388/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import random
sample_indices = random.choices(range(len(train_set)), k=5)

for sample in train_set.select(sample_indices):
    print(sample, '\n\n')

{'관리번호': 'KO-JA-2020-FOLK-022556', '분야': '문화재/향토/K-FOOD', '한국어': '천생산의 통신바위로 가는 길목 골짜기에 약수터가 자리하고 있으므로 약샘골이라 한다.', '일본어': '天生山の通信岩路に向かう街角の谷間に、薬水場が位置しているため、薬泉谷という。', '한국어_어절수': 10, '일본어_글자수': 36, '길이_분류': 2, '출처': 'http://gumi.grandculture.net/gumi/toc/GC01200066', '수행기관': '플리토'} 


{'관리번호': 'KO-JA-2020-SOCI-089000', '분야': '사회/노동/복지', '한국어': '하지만, 김 실장은 "이번 최저임금위원회의 결정이 소득주도성장의 폐기 또는 포기를 의미하는 것으로 오해되지 않았으면 한다"고 강조했다.', '일본어': 'だが、キム室長は「今回の最低賃金委員会の決定が所得主導成長の廃棄または放棄を意味すると誤解されないように望む」と強調した。', '한국어_어절수': 16, '일본어_글자수': 57, '길이_분류': 3, '출처': None, '수행기관': '플리토'} 


{'관리번호': 'KO-JA-2020-MEDI-045556', '분야': '의료/보건', '한국어': "확진자들이 대거 발생할 당시 '자쿠와' 술집 인근에 있었다고 밝힌 인원만 지난 21일 현재 1,000여 명에 달해 지역감염 확산 및 '제2의 이태원 클럽'이 되는 것 우려의 목소리가 커지고 있다.", '일본어': '多くの感染者が発生した当時、「ザクワ」居酒屋の付近にいたと明らかにした人数だけで21日現在約1,000人に達し、地域感染の拡散及び「第2の梨泰院クラブ」になるのではないかと懸念する声が高まっている。', '한국어_어절수': 28, '일본어_글자수': 84, '길이_분류': 5, '출처': 'http://uci.or.kr/G703:RA101-01101101.20200523050227002:1', '수행기관': '솔트룩스파트너스'} 


{'관리번호': 'KO-JA-2020-FO

In [5]:
from transformers import AutoModel, AutoTokenizer, MT5Tokenizer, GPT2LMHeadModel, AutoConfig

ENCODER_MODEL = "bert-base-multilingual-cased"
DECODER_MODEL = "THUMT/mGPT"

mBERT = AutoModel.from_pretrained(ENCODER_MODEL)
mBERT_tokenizer = AutoTokenizer.from_pretrained(ENCODER_MODEL)

decoder_config = AutoConfig.from_pretrained(DECODER_MODEL)
decoder_config.add_cross_attention=True
mGPT = GPT2LMHeadModel.from_pretrained(DECODER_MODEL, config=decoder_config)
mGPT_tokenizer = MT5Tokenizer.from_pretrained(DECODER_MODEL)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at THUMT/mGPT and are newly initialized: ['transformer.h.3.ln_cro

In [ ]:
def func(examples, max_length):
    model_inputs = mBERT_tokenizer(examples['한국어'], max_length, padding="max_length", truncation=True)
    labels = mGPT_tokenizer(examples['일본어'], max_length=max_target_length, padding="max_target_length", truncation=True)
    pass

fn_kwargs = {"max_length": 512}
tokenized_train = train_set.map(func, num_proc=5, remove_columns=train_set.column_names, fn_kwargs=kn_kwargs)

In [ ]:
import random
sample_indices = random.choices(range(len(train_set)), k=5)

for sample in train_set.select(sample_indices):
    print(sample, '\n\n')

In [6]:
mGPT

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(250100, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (crossattention): GPT2Attention(
          (c_attn): Conv1D()
          (q_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_cross_attn): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, i

In [ ]:
print(mBERT_tokenizer.tokenize("이순신은조선중기의무신이다."))
print(mBERT_tokenizer.tokenize("아버지가방에들어가신다."))
print(mBERT_tokenizer.tokenize("被酷禍物を加熱したあと、水蒸気を噴射して酷禍させる配置方式の酷禍炉において、水蒸気に含まれた凝縮水などの不純物が、酷禍不良のみならず、脆弱な被酷禍物を損傷させることになる。"))
print()
print(mGPT_tokenizer.tokenize("이순신은조선중기의무신이다."))
print(mGPT_tokenizer.tokenize("아버지가방에들어가신다."))
print(mGPT_tokenizer.tokenize("被酷禍物を加熱したあと、水蒸気を噴射して酷禍させる配置方式の酷禍炉において、水蒸気に含まれた凝縮水などの不純物が、酷禍不良のみならず、脆弱な被酷禍物を損傷させることになる。"))